### Step 1. Building protein target profiles ###
The web services offered by ChEMBL (Davies et al., 2015) were used in the workflow to extract drug target protein data. </br>
At first, for each inputted drug, an assay search was performed across the database to query all proteins showing bioactivity value of pChEMBL 5 or above. </br>
pChEMBL is defined as: −log10 (molar IC50, XC50, EC50, AC50, Ki, Kd or Potency) (Bento et al., 2014). 

In [12]:
import pandas as pd
from chembl_webresource_client.new_client import new_client
import requests
import json

# List of ChEMBL IDs 
chembl_ids = ['CHEMBL112', 'CHEMBL185'] #example, please fill it with your compounds of interest 

# Initialize the new client for bioactivity queries
bioactivity = new_client.activity

# Set the minimum pChEMBL value for filtering (can be adjusted)
min_pchembl_value = 5.0

# Create an empty list to store results
bioactivity_data = []

# Loop through each ChEMBL ID and retrieve bioactivities
for chembl_id in chembl_ids:
    activities = bioactivity.filter(molecule_chembl_id=chembl_id)
    
    for activity in activities:
        # Check if pChEMBL value exists, convert it to float, and apply filtering
        pchembl_value = activity.get('pchembl_value')
        if pchembl_value is not None:
            pchembl_value = float(pchembl_value)  # Convert to float
            if pchembl_value >= min_pchembl_value:
                target_organims = activity.get('target_organism')
                if target_organims == 'Homo sapiens':
                # Store relevant bioactivity information
                    bioactivity_data.append({
                        'Compound ChEMBL ID': chembl_id,
                        'Assay ChEMBL ID': activity.get('assay_chembl_id'),
                        'Bioactivity Type': activity.get('type'),
                        'Activity Value': activity.get('value'),
                        'Activity Units': activity.get('units'),
                        'pChEMBL Value': pchembl_value,
                        'Target ChEMBL ID': activity.get('target_chembl_id'),
                        'Target Organism': activity.get('target_organism'),
                        'Assay Description': activity.get('assay_description')
                        })

# Convert the list of bioactivities to a pandas DataFrame
bioactivity_df = pd.DataFrame(bioactivity_data)

# Ensure the 'pChEMBL Value' column is of float type
bioactivity_df['pChEMBL Value'] = bioactivity_df['pChEMBL Value'].astype(float)

# Display the DataFrame
print(bioactivity_df)

# Optionally, you can save the DataFrame to a CSV file
#bioactivity_df.to_csv('filtered_bioactivities.csv', index=False)



    Compound ChEMBL ID Assay ChEMBL ID Bioactivity Type Activity Value  \
0            CHEMBL112    CHEMBL933689               Ki           10.0   
1            CHEMBL112    CHEMBL933690               Ki            6.2   
2            CHEMBL112    CHEMBL992759               Ki            4.1   
3            CHEMBL112    CHEMBL992757               Ki            9.1   
4            CHEMBL112    CHEMBL991857               Ki            7.1   
..                 ...             ...              ...            ...   
805          CHEMBL185   CHEMBL5249187             IC50           5.78   
806          CHEMBL185   CHEMBL5249191             IC50           5.05   
807          CHEMBL185   CHEMBL5249192             IC50            2.5   
808          CHEMBL185   CHEMBL5249193             IC50           5.81   
809          CHEMBL185   CHEMBL5258495             IC50           5.31   

    Activity Units  pChEMBL Value Target ChEMBL ID Target Organism  \
0               uM           5.00        

### Step 2. Unifying the list of protein targets ###
The collected drug-protein pairs from each database were harmonized and collated, resulting in a list with ChEMBL ID for drugs and UniProt accession number for target proteins. </br>
It was ensured, that the final list only contains reviewed human proteins, and the duplicate drug-protein pairs were removed.

In [2]:
# List of ChEMBL target IDs to translate
chembl_target_ids = bioactivity_df['Target ChEMBL ID']  # Example list

# Initialize the new client for target queries
target_client = new_client.target

# Create an empty list to store results
target_data = []

# Loop through each ChEMBL target ID and retrieve the UniProt ID
for target_id in chembl_target_ids:
    target_info = target_client.get(target_id)
    
    # Retrieve the UniProt IDs from the target components
    uniprot_ids = []
    if 'target_components' in target_info:
        for component in target_info['target_components']:
            if 'accession' in component:
                uniprot_ids.append(component['accession'])
    
    # Store the ChEMBL target ID and associated UniProt IDs
    target_data.append({
        'ChEMBL Target ID': target_id,
        'UniProt IDs': ', '.join(uniprot_ids) if uniprot_ids else 'N/A'
    })

# Convert the list of targets to a pandas DataFrame
target_df = pd.DataFrame(target_data)

# Display the DataFrame
print(target_df)

# Optionally, you can save the DataFrame to a CSV file
#target_df.to_csv('chembl_to_uniprot.csv', index=False)

# Rename columns 
target_df = target_df.rename(columns={'ChEMBL Target ID': 'Target ChEMBL ID'})

# Merge everything together in "merged_df"
for index, row in target_df.iterrows():
    if pd.notna(row["UniProt IDs"]):
        merged_df = pd.merge(target_df, bioactivity_df, on = 'Target ChEMBL ID', how = 'inner' )

#Keep the relevant columns
merged_relevant_df = merged_df[[ 'Compound ChEMBL ID', 'UniProt IDs',]]

# Remove duplicate rows and rows where there is no UniProt ID available
merged_relevant_df = merged_relevant_df.drop_duplicates()
merged_relevant_df = merged_relevant_df[merged_relevant_df['UniProt IDs'] != 'N/A']

# Display the DataFrame without duplicates
merged_relevant_df



    ChEMBL Target ID UniProt IDs
0          CHEMBL261      P00915
1          CHEMBL205      P00918
2         CHEMBL3242      O43570
3         CHEMBL2326      P43166
4         CHEMBL2885      P07451
..               ...         ...
805        CHEMBL399         N/A
806        CHEMBL387         N/A
807        CHEMBL390         N/A
808    CHEMBL3706566         N/A
809        CHEMBL384         N/A

[810 rows x 2 columns]


,Compound ChEMBL ID,UniProt IDs
0,CHEMBL112,P00915
9,CHEMBL112,P00918
18,CHEMBL112,O43570
19,CHEMBL112,P43166
20,CHEMBL112,P07451
21,CHEMBL112,P16473
22,CHEMBL185,P16473
30,CHEMBL112,P02144
28335,CHEMBL185,P02545
28351,CHEMBL185,P42858


### Step 3. Tissue-specific target profiles ###
Optional filtering, which removes protein targets not expressed in a tissue, was added to the workflow to support the analysis of a specific tissue type. <br/>
The consensus mRNA expression from The Tissue Atlas of The Human Protein Atlas (https://www.proteinatlas.org); (Uhlén et al., 2015) was used to build this filter,<br/>
which is a merged dataset from three sources.

In [5]:

expression = pd.read_csv('rna_tissue_consensus.tsv', sep='\t')

# UniProt IDs that we want to have the Gene Name for
accessions = merged_relevant_df["UniProt IDs"]

# Function to get gene name from EBI Protein API
def get_gene_name(accession):
    url = f"https://www.ebi.ac.uk/proteins/api/proteins/{accession}?fields=gene"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        gene_info = data.get("gene")
        if gene_info:
            return gene_info[0].get("name", {}).get("value")
        else:
            return "No gene info"
    else:
        return "Request failed"

# Apply the function to the 'UniProt IDs' column and create a new 'Gene name' column
merged_relevant_df['Gene name'] = merged_relevant_df['UniProt IDs'].apply(get_gene_name)

# Display the updated dataframe
#print(merged_relevant_df)



In [6]:
# Incorporae the expresssion data
for index, row in merged_relevant_df.iterrows():
    if pd.notna(row["UniProt IDs"]):
        expression_df = pd.merge(merged_relevant_df, expression, on = 'Gene name', how = 'inner' )

# Display the expression dataframe
expression_df.head()

,Compound ChEMBL ID,UniProt IDs,Gene name,Gene,Tissue,nTPM
0,CHEMBL112,P00915,CA1,ENSG00000133742,adipose tissue,2.0
1,CHEMBL112,P00915,CA1,ENSG00000133742,adrenal gland,0.4
2,CHEMBL112,P00915,CA1,ENSG00000133742,amygdala,0.9
3,CHEMBL112,P00915,CA1,ENSG00000133742,appendix,0.5
4,CHEMBL112,P00915,CA1,ENSG00000133742,basal ganglia,0.9


In [7]:
#Modify according to the tissue expression of interest (here, liver)
liver_expression_df = expression_df[(expression_df["Tissue"] == "liver") & (expression_df["nTPM"] >= 11)]

# Display the filtered dataframe
liver_expression_df


,Compound ChEMBL ID,UniProt IDs,Gene name,Gene,Tissue,nTPM
72,CHEMBL112,P00918,CA2,ENSG00000104267,liver,265.1
422,CHEMBL185,P02545,LMNA,ENSG00000160789,liver,90.9
522,CHEMBL185,Q16637,SMN1,ENSG00000172062,liver,36.6
622,CHEMBL185,Q01453,PMP22,ENSG00000109099,liver,11.5
672,CHEMBL185,P40225,THPO,ENSG00000090534,liver,34.5


### Step 4. Connecting drugs and biological pathways ###

With the list of tissue-specific target proteins for each drug, a statistical overrepresentation test was performed through the Reactome pathway analysis API service (Fabregat et al., 2017).

In [8]:
# creating the request body based on the DataFrame containing series (lists) of UniProt IDs

request_body  = liver_expression_df.groupby('Compound ChEMBL ID')['UniProt IDs'].apply(list).reset_index()
request_body= pd.DataFrame(request_body)
request_body_API = pd.DataFrame(request_body['UniProt IDs'])



In [9]:

# Reactome API endpoint with query parameters
url = "https://reactome.org/AnalysisService/identifiers/?interactors=false&pageSize=20&page=1&sortBy=ENTITIES_PVALUE&order=ASC&resource=TOTAL&pValue=1&includeDisease=false"

# Set headers for the POST request
headers = {
    'Content-Type': 'text/plain'  # Use text/plain for sending raw data
}

# Define timeout in seconds (e.g., 10 seconds)
timeout_duration = 10

# Create an empty list to store pathway data for all UniProt ID series
all_pathway_data = []

# Iterate over each series of UniProt IDs in the DataFrame
for index, row in request_body_API.iterrows():
    # Convert the list of UniProt IDs into a comma-separated string
    uniprot_ids = ','.join(row['UniProt IDs'])
    
    try:
        # Send the POST request with the list of UniProt IDs
        response = requests.post(url, headers=headers, data=uniprot_ids, timeout=timeout_duration)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response JSON
            pathways = response.json()

            # Extract 'pathway name' and 'identifier' for this series of UniProt IDs
            for result in pathways.get('pathways', []):  # Adjust this depending on the JSON structure
                pathway_name = result.get('name', 'N/A')
                identifier = result.get('stId', 'N/A')  # 'stId' is usually the pathway identifier
                
                # Store the UniProt series and associated pathway info
                all_pathway_data.append({
                    'UniProt IDs': uniprot_ids,
                    'Pathway Name': pathway_name,
                    'Identifier': identifier
                })
        
        else:
            # Log the error if the request was not successful
            print(f"Error: {response.status_code}, {response.text}")

    except requests.Timeout:
        print(f"The request timed out after {timeout_duration} seconds for UniProt IDs: {uniprot_ids}")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")

# Convert the list of pathway data into a DataFrame
df_pathways = pd.DataFrame(all_pathway_data)

# Print the final DataFrame with UniProt IDs, Pathway Names, and Identifiers
print(df_pathways)






                    UniProt IDs  \
0                        P00918   
1                        P00918   
2                        P00918   
3                        P00918   
4                        P00918   
5                        P00918   
6   P02545,Q16637,Q01453,P40225   
7   P02545,Q16637,Q01453,P40225   
8   P02545,Q16637,Q01453,P40225   
9   P02545,Q16637,Q01453,P40225   
10  P02545,Q16637,Q01453,P40225   
11  P02545,Q16637,Q01453,P40225   
12  P02545,Q16637,Q01453,P40225   
13  P02545,Q16637,Q01453,P40225   
14  P02545,Q16637,Q01453,P40225   
15  P02545,Q16637,Q01453,P40225   
16  P02545,Q16637,Q01453,P40225   
17  P02545,Q16637,Q01453,P40225   
18  P02545,Q16637,Q01453,P40225   
19  P02545,Q16637,Q01453,P40225   
20  P02545,Q16637,Q01453,P40225   
21  P02545,Q16637,Q01453,P40225   
22  P02545,Q16637,Q01453,P40225   
23  P02545,Q16637,Q01453,P40225   
24  P02545,Q16637,Q01453,P40225   
25  P02545,Q16637,Q01453,P40225   

                                         Pathway Name 

In [10]:
# Convert lists in 'UniProt IDs' in df1 to comma-separated strings
request_body['UniProt IDs'] = request_body['UniProt IDs'].apply(lambda x: ','.join(x))

# Merge the two DataFrames on the 'UniProt IDs' column
final_results_API = pd.merge(request_body, df_pathways, on='UniProt IDs', how='inner')

# Display the merged DataFrame
print(final_results_API)

#Optionally, you can save the final DataFrame to a CSV file
#final_results_API.to_csv('final_results_API.csv', index=False)


   Compound ChEMBL ID                  UniProt IDs  \
0           CHEMBL112                       P00918   
1           CHEMBL112                       P00918   
2           CHEMBL112                       P00918   
3           CHEMBL112                       P00918   
4           CHEMBL112                       P00918   
5           CHEMBL112                       P00918   
6           CHEMBL185  P02545,Q16637,Q01453,P40225   
7           CHEMBL185  P02545,Q16637,Q01453,P40225   
8           CHEMBL185  P02545,Q16637,Q01453,P40225   
9           CHEMBL185  P02545,Q16637,Q01453,P40225   
10          CHEMBL185  P02545,Q16637,Q01453,P40225   
11          CHEMBL185  P02545,Q16637,Q01453,P40225   
12          CHEMBL185  P02545,Q16637,Q01453,P40225   
13          CHEMBL185  P02545,Q16637,Q01453,P40225   
14          CHEMBL185  P02545,Q16637,Q01453,P40225   
15          CHEMBL185  P02545,Q16637,Q01453,P40225   
16          CHEMBL185  P02545,Q16637,Q01453,P40225   
17          CHEMBL185  P0254

In [59]:
final_results_API.to_csv('final_results_API.csv', index=False)